We call the training images supporting shots, and extracted SIFT feature points from them supporting points. These supporting points are in a fixed 128-dimensional space. Each feature point extracted from the test image is compared with all supporting points with a metric function, balanced by the difference in scale \(\sigma\). The K supporting points that are closest to the feature point extracted from the test image have the chance to vote to the class they belong to.

In [1]:
# Python 2
import cv2
import numpy as np
import pandas as pd
import random
from __future__ import print_function
from os import listdir
from os.path import join
from scipy.spatial.distance import cosine

In [2]:
img_dir = "data"
df = pd.DataFrame(filter(lambda x: ".JPG" in x, listdir(img_dir)), columns=["name"])
print(df.dtypes)
print(df.head())

name    object
dtype: object
           name
0  image001.JPG
1  image002.JPG
2  image003.JPG
3  image004.JPG
4  image006.JPG


In [3]:
df["class"] = np.repeat(np.linspace(0, 49, num=50), 4)
df["class"] = df["class"].astype("category")
print(df.dtypes)
print(df.head())

name       object
class    category
dtype: object
           name class
0  image001.JPG     0
1  image002.JPG     0
2  image003.JPG     0
3  image004.JPG     0
4  image006.JPG     1


In [4]:
split = ["train"] * 200
for i in range(0, 50):
    split[i * 4 + random.randint(0, 3)] = "test"
df["split"] = split
df["split"] = df["split"].astype("category")
print(df.dtypes)
print(df.head())

name       object
class    category
split    category
dtype: object
           name class  split
0  image001.JPG     0  train
1  image002.JPG     0  train
2  image003.JPG     0  train
3  image004.JPG     0   test
4  image006.JPG     1  train


In [5]:
df_train = df[df["split"] == "train"]
df_train_name = list(df_train["name"])  # TODO: series to sequencially numpy array?
df_train_class = list(df_train["class"])
sift = cv2.SIFT(nfeatures=20)
support = list()
for i in range(0, 150):
    progress = int(i / 150.0 * 1000) / 10.0
    print("{}: {}%".format(df_train_name[i], progress), end="\r")
    img = cv2.imread(join(img_dir, df_train_name[i]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(img, None)
    for d in des:
        support.append((tuple(d), df_train_class[i]))

In [6]:
df_test = df[df["split"] == "test"]
df_test_name = list(df_test["name"])
sift = cv2.SIFT(nfeatures=10)
predict = list()
for i in range(0, 50):
    progress = int(i / 50.0 * 1000) / 10.0
    print("{}: {}%".format(df_train_name[i], progress), end="\r")
    img = cv2.imread(join(img_dir, df_test_name[i]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(img, None)
    vote = np.zeros(50, dtype=np.uint16)
    for d in des:
        top_supports = sorted(support, key=lambda (k, v): cosine(k, d))[0:5]
        for (k, v) in top_supports:
            vote[int(v)] += 1
    predict.append(vote.argmax())

In [7]:
correct = 0
for i in range(0, 50):
    if predict[i] == i:
        correct += 1
print(predict)
print(correct)

[0, 1, 8, 3, 4, 45, 6, 7, 8, 9, 13, 11, 12, 13, 21, 15, 20, 17, 18, 19, 20, 21, 22, 25, 34, 13, 26, 27, 28, 29, 3, 31, 32, 13, 21, 35, 13, 22, 38, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
36
